In [ ]:
!pip install transformers torch scikit-learn pandas

In [ ]:
import torch
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

In [ ]:
df = pd.read_csv("/content/Reddit_Title.csv",
                 sep=";",
                 engine="python",
                 usecols=[0, 1])
df = df[['title', 'label']]
df = df.rename(columns={"title": "text"})
df.dropna(inplace=True)
df["label"] = df["label"].astype(int)

df.head()

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(
    list(X_train),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    list(X_test),
    truncation=True,
    padding=True,
    max_length=128
)

class StressDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = StressDataset(train_encodings, y_train)
test_dataset = StressDataset(test_encodings, y_test)

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
).to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained("stress_distilbert")
tokenizer.save_pretrained("stress_distilbert")


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict_stress(text):
    model.to(device)
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    stress_prob = probs[0][1].item()

    if stress_prob >= 0.7:
        return "Stressed"
    else:
        return "Not Stressed"

In [ ]:
print(predict_stress("I feel overwhelmed with work and exams"))
print(predict_stress("I am extremely stressed and anxious"))
print(predict_stress("I feel calm and relaxed today"))
print(predict_stress("Today was a good and happy day"))

In [ ]:
df['label'].value_counts(normalize=True)

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

st.set_page_config(page_title="Stress Detection", page_icon="🧠")

@st.cache_resource
def load_model():
    tokenizer = DistilBertTokenizerFast.from_pretrained("stress_distilbert")
    model = DistilBertForSequenceClassification.from_pretrained("stress_distilbert")
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

st.title("🧠 Stress Detection from Text")

text = st.text_area("Enter text")

if st.button("Detect Stress"):
    if text.strip() == "":
        st.warning("Please enter some text")
    else:
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )
        with torch.no_grad():
            outputs = model(**inputs)

        pred = torch.argmax(outputs.logits, dim=1).item()
        st.error("Stress Detected") if pred == 1 else st.success("No Stress Detected")

In [ ]:
!zip -r stress_distilbert.zip stress_distilbert

In [ ]:
from google.colab import files
files.download("stress_distilbert.zip")